In [ ]:
#This file aims to produce a real case scenario of an incident report
#The report will be wrote into a json file

In [1]:
import json
import random
from Incident import Incident

In [2]:
#First of all, I've to instanciate dictionaries of controls and incidents
controls = {
    #control_key : [control_name, estimated_time]
    1 : ["Information Backup", 6],
    2 : ["Capacity Management", 6],
    3 : ["Off-Site Equipment Security", 24],
    4 : ["Physical Entry Controls", 8],
    5 : ["Restriction Of Access To Information", 40],
    6 : ["Management Of Secret Information Authentication", 40],
    7 : ["Equipment Maintenance", 24],
    8 : ["Capacity Management", 8],
    9 : ["Clock Synchronization", 2],
    10 : ["Information Backups", 6],
    11 : ["Supply Facilities", 72],
    12 : ["Network Controls", 8]
}

In [3]:
threats = {
    #threat_key : [threat_name, control_key1, control_key2, ...]
    1 : ["Denial Of Service", 1, 2],
    2 : ["Theft", 3],
    3 : ["Social Engineering", 4, 6],
    4 : ["Monitoring Errors", 5, 6],
    5 : ["Failure Of Physical Or Logical Origin", 7],
    6 : ["System Crashes Due To Resource Exhaustion", 8],
    7 : ["Abuse Of Access Privileges", 5],
    8 : ["Configuration Errors", 9],
    9 : ["Maintenance Errors", 10],
    10 : ["Inadequate Temperature Or Humidity Conditions", 11],
    11 : ["Failure Of Communication Services", 12]
}

In [4]:
#Now I've to make the dataset (of 14 incidents)
incidents = []
for _ in range(14):
    threat_key = random.randint(1,11)
    threat_list = threats.get(threat_key)
    #if the list contains just the threat name, we have to choose another threat
    while(len(threat_list) < 2):
        threat_key = random.randint(1,11)
        threat_list = threats.get(threat_key)
    threat_name = threat_list[0]
    control_key = threat_list[random.randint(1,len(threat_list)-1)]
    control_list = controls[control_key]
    control_name = control_list[0]
    estimated_time = control_list[1]
    incident = Incident(threat_key, threat_name, control_key,control_name, estimated_time)
    incidents.append(incident)

for i in range(len(incidents)):
    print(incidents[i])

threat_id: 8, threat_name: Configuration Errors, control_id: 9, control_name: Clock Synchronization, estimated_time: 2
threat_id: 10, threat_name: Inadequate Temperature Or Humidity Conditions, control_id: 11, control_name: Supply Facilities, estimated_time: 72
threat_id: 2, threat_name: Theft, control_id: 3, control_name: Off-Site Equipment Security, estimated_time: 24
threat_id: 6, threat_name: System Crashes Due To Resource Exhaustion, control_id: 8, control_name: Capacity Management, estimated_time: 8
threat_id: 4, threat_name: Monitoring Errors, control_id: 6, control_name: Management Of Secret Information Authentication, estimated_time: 40
threat_id: 8, threat_name: Configuration Errors, control_id: 9, control_name: Clock Synchronization, estimated_time: 2
threat_id: 8, threat_name: Configuration Errors, control_id: 9, control_name: Clock Synchronization, estimated_time: 2
threat_id: 7, threat_name: Abuse Of Access Privileges, control_id: 5, control_name: Restriction Of Access To

In [7]:
#Now I've to write a json string into a file to communicate the report to the QUBO Matrix maker
json_string = json.dumps([incident.dump() for incident in incidents], indent=3)
#print(json_string)

[
   {
      "threat_id": 8,
      "threat_name": "Configuration Errors",
      "control_id": 9,
      "control_name": "Clock Synchronization",
      "estimated_time": 2
   },
   {
      "threat_id": 10,
      "threat_name": "Inadequate Temperature Or Humidity Conditions",
      "control_id": 11,
      "control_name": "Supply Facilities",
      "estimated_time": 72
   },
   {
      "threat_id": 2,
      "threat_name": "Theft",
      "control_id": 3,
      "control_name": "Off-Site Equipment Security",
      "estimated_time": 24
   },
   {
      "threat_id": 6,
      "threat_name": "System Crashes Due To Resource Exhaustion",
      "control_id": 8,
      "control_name": "Capacity Management",
      "estimated_time": 8
   },
   {
      "threat_id": 4,
      "threat_name": "Monitoring Errors",
      "control_id": 6,
      "control_name": "Management Of Secret Information Authentication",
      "estimated_time": 40
   },
   {
      "threat_id": 8,
      "threat_name": "Configuration Errors

In [8]:
json_file = open("incidents_report.txt", "a")
json_file.write(json_string)
json_file.close()